Прошу дать еще немного времени

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [137]:
data = pd.read_csv('/Users/evgeniya/Desktop/Рекомендательные системы/2/data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [138]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [139]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [140]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights.iloc[:,0])
    probability = np.array(items_weights.iloc[:,1])
    recs = np.random.choice(items, size=n, replace=True, p=probability)
    
    return recs.tolist()

Сделайте предсказания

In [141]:
%%time

sales_log = data_train.groupby('item_id')['sales_value'].sum().reset_index()    

sales_log['sales_value'] = np.log(sales_log['sales_value'], out=np.zeros_like(sales_log['sales_value']), where=(sales_log['sales_value'] !=0))

sales_log.loc[sales_log['sales_value'] < 0, 'sales_value'] = 0

sales_log['sales_value'] = sales_log['sales_value']/(sales_log['sales_value'].sum())

result['weighted_random_recs_1'] = result['user_id'].apply(lambda x: weighted_random_recommendation(sales_log, n=5))

CPU times: user 1.77 s, sys: 23.7 ms, total: 1.79 s
Wall time: 1.79 s


In [142]:
%%time

sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
sales['sales_value'] = sales['sales_value']/(sales['sales_value'].sum())

result['weighted_random_recs_2'] = result['user_id'].apply(lambda x: weighted_random_recommendation(sales, n=5))

CPU times: user 1.73 s, sys: 4.12 ms, total: 1.73 s
Wall time: 1.73 s


In [143]:
result.head(5)

,user_id,actual,weighted_random_recs_1,weighted_random_recs_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[907892, 1038448, 1100066, 853041, 1122591]","[970747, 925112, 12301100, 879393, 926905]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534482, 8358895, 13115500, 1036426, 1004895]","[1130729, 957063, 1117393, 901470, 906646]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[7472368, 1002934, 2177947, 3683040, 914697]","[909894, 966718, 865456, 6534178, 835098]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[941178, 860490, 15717269, 10356077, 974466]","[937210, 941797, 10149640, 7409579, 6533936]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13095415, 5571280, 8203500, 1132478, 1107546]","[1075505, 1129049, 1048257, 1041796, 845208]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [144]:
result = pd.read_csv('preds.csv')# закгрузка predict с семианара
result.head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'preds.csv'

In [87]:
type(result['actual'][0])

str

In [1]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result[a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

NameError: name 'result' is not defined

In [84]:
result.apply(lambda x: precision_at_k(x['popular_recommendations'], x['actual'],  5), axis=1).mean()

0.15523996082272082

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [ ]:
# your_code

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.